<a href="https://colab.research.google.com/github/sumankmaiti/All-in-one/blob/main/device_fingerprinting_trial_4_as_in_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import model_from_json
import pickle
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import keras
!pip install keras_metrics
import keras_metrics

In [ ]:
# Load the data
active_data = pd.read_csv('/content/drive/MyDrive/working directory/signature_train.csv', header=None)
print("original active dataset: \n", active_data)

original active dataset: 
       0    1    2    3    4    5    ...  296  297  298  299   300          301
0     164   85    7    2    0   10  ...    0   10   20    3  icmp    10.42.0.1
1     137   33   67    1   46   20  ...    0    0    0    1  icmp    10.42.0.1
2     344   44    7  120   35   79  ...    0    0    0    1  icmp    10.42.0.1
3     227   77    3    0   20   69  ...    0    0    0    1  icmp    10.42.0.1
4      45  238   85    0    0   12  ...    0    0    0    1  icmp    10.42.0.1
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...          ...
6021   12    5    5    8    2    1  ...    0    0    0    1  icmp  10.42.0.211
6022    1    0    0    0    1    1  ...    0    0    0    1  icmp  10.42.0.211
6023    5    4    2    1    1    2  ...    0    0    0    1  icmp  10.42.0.211
6024   13    4    8    5    5    3  ...    0    0    0    1  icmp  10.42.0.211
6025    1    0    0    0    3    2  ...    0    0    0    1  icmp  10.42.0.211

[6026 rows x 302 columns

In [ ]:
# per traffic type dataset
traffic_types = active_data.iloc[:, 300].unique()
print("traffic types are", traffic_types)
classes = active_data.iloc[:, 301].unique()
print("no of devices are", len(classes), classes)

traffic types are ['icmp' 'udp']
no of devices are 5 ['10.42.0.1' '10.42.0.17' '10.42.0.127' '10.42.0.211' '10.42.0.157']


In [ ]:
# for each traffic type a dataset will be stored
dataset = []
for items in range(len(traffic_types)):
    d = active_data[active_data[300] == traffic_types[items]]
    dataset.append(d)

print(dataset[0])

      0    1    2    3    4    5    ...  296  297  298  299   300          301
0     164   85    7    2    0   10  ...    0   10   20    3  icmp    10.42.0.1
1     137   33   67    1   46   20  ...    0    0    0    1  icmp    10.42.0.1
2     344   44    7  120   35   79  ...    0    0    0    1  icmp    10.42.0.1
3     227   77    3    0   20   69  ...    0    0    0    1  icmp    10.42.0.1
4      45  238   85    0    0   12  ...    0    0    0    1  icmp    10.42.0.1
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...          ...
6021   12    5    5    8    2    1  ...    0    0    0    1  icmp  10.42.0.211
6022    1    0    0    0    1    1  ...    0    0    0    1  icmp  10.42.0.211
6023    5    4    2    1    1    2  ...    0    0    0    1  icmp  10.42.0.211
6024   13    4    8    5    5    3  ...    0    0    0    1  icmp  10.42.0.211
6025    1    0    0    0    3    2  ...    0    0    0    1  icmp  10.42.0.211

[6024 rows x 302 columns]


In [ ]:
# input to NN
x = []
for item in dataset:
    temp = item.iloc[:, :300]
    x.append(temp)
print("input: \n", x[0])

input: 
       0    1    2    3    4    5    6    ...  293  294  295  296  297  298  299
0     164   85    7    2    0   10   30  ...    0    0    0    0   10   20    3
1     137   33   67    1   46   20   39  ...    0    0    0    0    0    0    1
2     344   44    7  120   35   79   43  ...    0    0    0    0    0    0    1
3     227   77    3    0   20   69   88  ...    0    0    0    0    0    0    1
4      45  238   85    0    0   12  153  ...    0    0    0    0    0    0    1
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
6021   12    5    5    8    2    1    0  ...    0    1    0    0    0    0    1
6022    1    0    0    0    1    1    0  ...    0    0    0    0    0    0    1
6023    5    4    2    1    1    2   10  ...    0    0    0    0    0    0    1
6024   13    4    8    5    5    3    2  ...    0    1    0    0    0    0    1
6025    1    0    0    0    3    2    1  ...    0    0    0    0    0    0    1

[6024 rows x 300 columns]


In [ ]:
# label
y = []
for item in dataset:
    temp = item.iloc[:, 301:]
    y.append(temp)
print("labels:\n", y[0])

labels:
               301
0       10.42.0.1
1       10.42.0.1
2       10.42.0.1
3       10.42.0.1
4       10.42.0.1
...           ...
6021  10.42.0.211
6022  10.42.0.211
6023  10.42.0.211
6024  10.42.0.211
6025  10.42.0.211

[6024 rows x 1 columns]


In [ ]:
# split dataset to train test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x[0], y[0], test_size=0.3, random_state=64)

print("xtrain data:\n", Xtrain)
print("ytrain data:\n", Ytrain)

xtrain data:
       0    1    2    3    4    5    6    ...  293  294  295  296  297  298  299
1438   93   11   54    3    8   45   15  ...    0    0    0    0    0    0    1
572    30   11    3    3    2    9   18  ...    0    0    0    0    0    0    1
646     5    0   10   14    6    4    2  ...    0    0    0    0    0    0    1
6011   16    2    6    2    1    1    5  ...    0    0    0    0    0    0    1
3731   32   75  118    4   52   27    0  ...    0    0    0    0    0    0    1
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
5110    1    0    0    1    3    1    0  ...    0    0    0    0    0    0    1
1009    2    0    0    2    0    5    0  ...    0    0    0    0    0    0    1
54    112   79   93  307    4    2    6  ...    0    0    0    0    1  128   61
3240  115   70   50   38   37   45   53  ...    0    0    0    0    0    0    1
5096  384   22    1   81   23    0    1  ...    0    0    0    0    0    0    1

[4216 rows x 300 columns]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Xnorm = scaler.fit_transform(Xtrain)
Xnorm_test = scaler.transform(Xtest)
print("after normalization of inputs: \n", Xnorm)

after normalization of inputs: 
 [[0.0685544  0.0248307  0.08307692 ... 0.         0.         0.        ]
 [0.02160954 0.0248307  0.00461538 ... 0.         0.         0.        ]
 [0.00298063 0.         0.01538462 ... 0.         0.         0.        ]
 ...
 [0.08271237 0.17832957 0.14307692 ... 0.01298701 0.55411255 0.52631579]
 [0.08494784 0.15801354 0.07692308 ... 0.         0.         0.        ]
 [0.28539493 0.0496614  0.00153846 ... 0.         0.         0.        ]]


In [ ]:
# convert IDs to integer
encoder = LabelEncoder()
y_dash = encoder.fit_transform(Ytrain)
print(y_dash.shape)
ytr = to_categorical(y_dash)
print("after encoding ids: \n", ytr, "\n", ytr.shape)

(4216,)
after encoding ids: 
 [[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]] 
 (4216, 5)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_dash = encoder.transform(Ytest)
print(y_dash.shape)
yts = to_categorical(y_dash)
print("after encoding ids: \n", yts, "\n", yts.shape)

(1808,)
after encoding ids: 
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]] 
 (1808, 5)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# ANN model
model = Sequential()
model.add(Dense(50, activation='sigmoid', input_dim=300))
model.add(Dense(len(classes), activation='sigmoid'))
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall(), 'accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(Xnorm, ytr, epochs=500, batch_size=4, validation_split=0.4)

Epoch 1/500
633/633 [==============================] - 2s 3ms/step - loss: 0.4064 - accuracy: 0.5049 - val_loss: 0.3597 - val_accuracy: 0.5086
Epoch 2/500
633/633 [==============================] - 2s 3ms/step - loss: 0.3237 - accuracy: 0.5730 - val_loss: 0.3041 - val_accuracy: 0.5886
Epoch 3/500
633/633 [==============================] - 2s 3ms/step - loss: 0.2895 - accuracy: 0.6153 - val_loss: 0.2843 - val_accuracy: 0.6147
Epoch 4/500
633/633 [==============================] - 2s 3ms/step - loss: 0.2760 - accuracy: 0.6287 - val_loss: 0.2759 - val_accuracy: 0.6183
Epoch 5/500
633/633 [==============================] - 1s 2ms/step - loss: 0.2686 - accuracy: 0.6414 - val_loss: 0.2694 - val_accuracy: 0.6224
Epoch 6/500
633/633 [==============================] - 1s 2ms/step - loss: 0.2633 - accuracy: 0.6493 - val_loss: 0.2673 - val_accuracy: 0.6236
Epoch 7/500
633/633 [==============================] - 2s 2ms/step - loss: 0.2601 - accuracy: 0.6489 - val_loss: 0.2628 - val_accuracy: 0.6349

In [ ]:

     

    
    
    acc = history.history['accuracy']
    print("train accuracy:", acc[len(acc)-1])
    filename = 'accuracy'+str(i)+'.sav'
    pickle.dump(acc[len(acc)-1], open(filename, 'wb'))
    
    acc = history.history['val_accuracy']
    print("validation accuracy:", acc[len(acc)-1])
    filename = 'val_accuracy'+str(i)+'.sav'
    pickle.dump(acc[len(acc)-1], open(filename, 'wb'))    
    
    acc = history.history['loss']
    print("train loss:", acc[len(acc)-1])
    filename = 'loss'+str(i)+'.sav'
    pickle.dump(acc[len(acc)-1], open(filename, 'wb'))
    
    acc = history.history['val_loss']
    print("validation loss:", acc[len(acc)-1])
    filename = 'val_loss'+str(i)+'.sav'
    pickle.dump(acc[len(acc)-1], open(filename, 'wb'))
    
    _, accur = model.evaluate(Xnorm_test, yts)
    filename = 'test_accuracy'+str(i)+'.sav'
    pickle.dump(accur, open(filename, 'wb')) 
    print("test accuracy:", accur)

    # save the model to disk
    model_json = model.to_json()
    with open("model"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model"+str(i)+".h5")
    #
accuracy = []
val_accuracy = []
loss = []
val_loss = []
test_accuracy = []

for i in range(10, 1000, 10):
    filename = "accuracy"+str(i)+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    accuracy.append(loaded_model)
    
    filename = "loss"+str(i)+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    loss.append(loaded_model)
    
    filename = "test_accuracy"+str(i)+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    test_accuracy.append(loaded_model)
    
    filename = "val_accuracy"+str(i)+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    val_accuracy.append(loaded_model)
    
    filename = "val_loss"+str(i)+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    val_loss.append(loaded_model)

    
# print("testing the model with test dataset:")
#
# test_data = pd.read_csv('signature_test.csv', header=None)
# print("original test dataset: \n", test_data)
#
# # per traffic type dataset
# test_traffic_types = test_data.iloc[:, 300].unique()
# print("traffic types are", test_traffic_types)
# test_classes = test_data.iloc[:, 301].unique()
# print("no of devices are", len(test_classes), test_classes)
#
# # for each traffic type a dataset will be stored
# test_dataset = []
# for items in range(len(test_traffic_types)):
#     d = test_data[test_data[300] == test_traffic_types[items]]
#     test_dataset.append(d)

# print(test_dataset[0])

# input to NN
# test_x = []
# for item in test_dataset:
#     temp = item.iloc[:, :300]
#     test_x.append(temp)
# print("input: \n", test_x[0])
#
# # label
# test_y = []
# for item in test_dataset:
#     temp = item.iloc[:, 301:]
#     test_y.append(temp)
# print("labels:\n", test_y[0])
#
# test_xnorm = scaler.transform(test_x[0])
# print(test_xnorm)
#
# y_dash = encoder.transform(test_y[0])
# print(y_dash.shape)
# test_yts = to_categorical(y_dash)
# print("after encoding ids: \n", test_yts, "\n", test_yts.shape)
#
# pred = model.predict(test_xnorm)
# test_class = np.argmax(test_yts, axis=1)
# pred_class = np.argmax(pred, axis=1)



# # print("confusion matrix:\n ", confusion_matrix(test_class, pred_class), "\n", classification_report(test_class, pred_class))
#
# print("actual classes:\n", test_class)
# print("predicted  classes\n", pred_class)
#
# # z = encoder.inverse_transform(test_class)
# # print("original ID of 11th device in dataset:", z[11])
# # z = encoder.inverse_transform(pred_class)
# # print("predicted ID of 11th device in dataset:", z[11])
# #
# #
# # # saving the trained neural network
# # model_json = model.to_json()
# # with open("model.json", "w") as json_file:
# #     json_file.write(model_json)
# #
# # model.save_weights("model.h5")
#
# ############################# Testing #####################################
# print("\n\n Testing from here")
# # Load the data
# data = pd.read_csv('signature_with_ID(testing).csv', header=None)
# print("original dataset: \n", data)
#
# # input to NN
# Xt = data.iloc[:, :300]
# # print("input: \n", Xt)
#
# # label
# Yt = data.iloc[0:, 300:]
# # print("label:\n", Yt)
#
# Xtnorm = scaler.transform(Xt)
#
# # print("after normalization of testing inputs: \n", Xtnorm)
#
# pred = model.predict(Xtnorm)
# # test_class = np.argmax(Ytest, axis=1)
# pred_class = np.argmax(pred, axis=1)
#
# z = encoder.inverse_transform(pred_class)
# print("predicted ID of 11th device in dataset:", z)
